In [ ]:
# function
import os
import sys

new_directory = "E:/airflow/airflow"
current_directory = os.getcwd()
scripts_path = os.path.abspath(os.path.join(os.getcwd(), '../scripts'))

def change_directory(current_directory, new_directory,scripts_path):
    # Get the current working directory
    print(f'Current directory: {current_directory}')
    # Define the path to change to
    new_directory = "E:/airflow/airflow"
    try:
        # Change the current working directory
        os.chdir(new_directory)
        # Verify the change
        current_directory = os.getcwd()
        print(f'Current directory changed to: {current_directory}')
    except FileNotFoundError:
        print(f'Error: The directory "{new_directory}" does not exist.')
    except PermissionError:
        print(f'Error: Permission denied to change to "{new_directory}".')
    except Exception as e:
        print(f'An unexpected error occurred: {e}')
    # Add the scripts directory to the Python path
    sys.path.append(scripts_path)

    
change_directory(current_directory, new_directory, scripts_path)

In [ ]:
import pandas as pd
import warnings
import sklearn
from sklearn.preprocessing import StandardScaler
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [ ]:
# Setting up all directory
root_folder = new_directory
database_path = root_folder+"/database/"
data_directory = root_folder+"/data/raw/"
data_profile_path = root_folder+"/data/profile_report/"
intermediate_data_path = root_folder+"/data/interim/"
final_processed_data_path = root_folder+"/data/processed/"

old_data_directory = root_folder+"/data/raw/"
new_data_directory = root_folder+"/data/new/"
intermediate_path = root_folder+"/data/interim/"


# Database
db_path = root_folder+"/database/"
db_file_name = "feature_store_v01.db"
drfit_db_name = "drift_db_name.db"
date_columns = ['registration_init_time','transaction_date_min','transaction_date_max','membership_expire_date_max','last_login']
drift_db_name = "drift_db_name.db"

# Mlflow
mlflow_tracking_uri = "http://Localhost:6006"
ml_flow_model_path = root_folder+ "/mlruns/2/cb66e22bcbf74ded99dc219eb29e7609/artifacts/models/"
ml_flow_path = root_folder+ "/mlruns/2/cb66e22bcbf74ded99dc219eb29e7609"

run_on = "old" #"old"
append=False
date_transformation = False
start_date = '2017-03-01'
end_date = '2017-03-31'

In [ ]:
new_data=pd.read_csv(root_folder + '/data/data_unseen.csv')

In [ ]:
new_data.drop(['msno'],axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

def get_data_prepared_for_modeling(dataframe, scale_method='standard', date_columns=None, corr_threshold=0.90):
    print("Initial columns:", len(dataframe.columns))
    
    # Step 1: Date Conversion
    if date_columns:
        for column in date_columns:
            try:
                dataframe[column] = pd.to_datetime(dataframe[column], errors='coerce')
            except Exception as e:
                print(f"Error processing {column}: {e}")
    
    # Step 2: Remove Multicollinearity
    # Create correlation matrix
    corr_matrix = dataframe.corr().abs()
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    # Find features with correlation greater than threshold
    to_drop = [column for column in upper.columns if any(upper[column] > corr_threshold)]
    print("Columns to drop due to high correlation:", to_drop)
    dataframe.drop(to_drop, axis=1, inplace=True)
    print("Columns after dropping:", len(dataframe.columns))
    
    # Step 3: Feature Engineering on Date Columns
    if date_columns:
        features = ["day", "month", "year", "weekday"]
        for eachcol in date_columns:
            if eachcol in dataframe.columns:
                # Ensure conversion
                dataframe[eachcol] = pd.to_datetime(dataframe[eachcol], errors='coerce')
                for eachfeature in features:
                    col_name = f"{eachcol}_{eachfeature}"
                    if eachfeature == 'day':
                        dataframe[col_name] = dataframe[eachcol].dt.day
                    elif eachfeature == 'month':
                        dataframe[col_name] = dataframe[eachcol].dt.month
                    elif eachfeature == 'year':
                        dataframe[col_name] = dataframe[eachcol].dt.year
                    elif eachfeature == 'weekday':
                        dataframe[col_name] = dataframe[eachcol].dt.weekday
        
        # Drop original date columns
        dataframe.drop(date_columns, axis=1, inplace=True)
    
    # Step 4: Scaling Numeric Features
    column_to_scale = dataframe.select_dtypes(include=['float64', 'int64']).columns.drop('is_churn')
    if scale_method == 'standard':
        transformer = StandardScaler().fit(dataframe[column_to_scale])
        scaled_data = pd.DataFrame(transformer.transform(dataframe[column_to_scale]), columns=column_to_scale)
    
    # Step 5: Combining Scaled and Other Features
    final_df = pd.concat([scaled_data, dataframe.drop(column_to_scale, axis=1)], axis=1)
    
    # Step 6: Splitting X, y
    X = final_df.drop(['is_churn'], axis=1)
    y = final_df[['is_churn']]
    
    return X, y


In [ ]:
X,y = get_data_prepared_for_modeling(new_data,date_columns = ['registration_init_time','transaction_date_min','transaction_date_max','membership_expire_date_max','last_login'])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import lightgbm as lgb

def get_train_model(X,y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
  #Model Training
  clf = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               device='gpu', importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
               objective=None, random_state=42, reg_alpha=0.0, reg_lambda=0.0,
               subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)
  clf.fit(X_train, y_train)
  # predict the results
  y_pred=clf.predict(X_test)
  # view accuracy
  accuracy=accuracy_score(y_pred, y_test)
  print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
  print(classification_report(y_test, y_pred))
  print(confusion_matrix(y_test, y_pred))
  return clf

In [ ]:
def get_predict(model,x):
  return model.predict_proba(x)

In [ ]:
# Full Pipeline 

#new data coming in with Y values 
#X,y = get_data_prepared_for_modeling(new_data,date_columns = ['registration_init_time','transaction_date_min','transaction_date_max','membership_expire_date_max','last_login'])

# Train the model with X & y 
model = get_train_model(X,y)

#Predict Object 
predictions = get_predict(model,X)

In [ ]:
import lightgbm as lgbm
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import StratifiedKFold

num_splits = 20
strat_kf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

scores = np.empty(num_splits)
for idx, (train_idx, test_idx) in enumerate(strat_kf.split(X, y)):
    print("=" * 10 + f"Training fold {idx}" + 10 * "=")

    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
    eval_set = [(X_val, y_val)]

    lgbm_clf = lgbm.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
               objective=None, random_state=42, reg_alpha=0.0, reg_lambda=0.0,
               subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)
    lgbm_clf.fit(X_train, y_train)
    # predict the results
    y_pred=lgbm_clf.predict(X_val)
    # view accuracy
    accuracy=accuracy_score(y_val, y_pred)
    print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy))
    print(classification_report(y_val, y_pred))
    print(confusion_matrix(y_val, y_pred))